# House Price Prediction


## Boosting

This notebook discusses the algorithm and application of boosting methodologies for better predicting performance than single models.

## Overview
- AdaBoost
    - Algorithm for Regression
- Gradient Boosting Model (GBM)
    - AdaBoost vs Gradient Boosting
    - General Form
    - Gradient Tree Boosting Algorithm
- XGBoost

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
### Read in the data
df = pd.read_csv('../project-house-price-prediction/data/train.csv')
df_test = pd.read_csv('../project-house-price-prediction/data/test.csv')

## Data
The data preprocessing follows the data exploration done in the notebook notebook data-exploration-and-preprocessing and 

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
class CleanHouseAttributes(BaseEstimator, TransformerMixin):
    """Apply rules during data exploration to clean house price dataset"""
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, df_house_price, cols_to_drop, target_col):
        # Age of building/remodle from YearBuilt and YearRemodAdd
        df_house_price['AgeBuilding'] = 2012 - df_house_price['YearBuilt']
        df_house_price['AgeRemodel'] = 2012 - df_house_price['YearRemodAdd']
    
        # Remove categories not exist in test.csv
        df_filtered = df_house_price[(df_house_price['HouseStyle'] != '2.5Fin') &
                                     (df_house_price['Exterior1st'] != 'Stone') &
                                     (df_house_price['Exterior1st'] != 'ImStucc') &
                                     (df_house_price['Exterior2nd'] != 'Other')]
        
        # Drop columns
        df_dropped = df_filtered.drop(cols_to_drop + target_col, axis=1)
            
        # Fill NA for numeric columns
        df_numeric = df_dropped.select_dtypes(include=['int64', 'float64']).apply(lambda x: x.fillna(x.mean()), axis=1)
        
        # Create dummies for non numeric columns
        df_nonNumeric = pd.get_dummies(df_dropped.select_dtypes(include=['object']).fillna('NA'))
        
        # Create boolean variables for Alley, PoolQC, and Fence
        df['HasAlley'] = list(1 if x is None else 0 for x in df['Alley'])
        df['HasPool'] = list(1 if x is None else 0 for x in df['PoolQC'])
        df['Fence'] = list(1 if x is None else 0 for x in df['Fence'])
        
        X = pd.concat([df_numeric, df_nonNumeric], axis=1)
        y = df_filtered[target_col]

        return X, y

In [ ]:
drop_cols = ['Id', 'YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'TotalBsmtSF', 
             'TotRmsAbvGrd', 'MoSold', 'YrSold', 'Street', 'Alley', 'Utilities', 
             'LandSlope', 'Condition2', 'Heating', 'Functional', 'FireplaceQu', 
             'PoolQC', 'Fence', 'MiscFeature'            ]
target_col = ['SalePrice']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

### AdaBoosting $_{[1]}$

In bagging, each training example is equally likely to be picked. In boosting, the probability of a particular example being in the training set of a particular machine depends on the performance of the prior machines on that sample. In each round, each training samples is assigned a new weight depending on the prediction performance from the previous round. More weights are assigned to samples with worse prediction so that each subsequent machine would focus on training the "difficult" samples.

#### Algorithm for Regression
Given $(x_1, y_1),...,(x_n, y_n)$, assign a weight $w_i=1$ for $i = 1,...,n$.

For $t = 1,..., T$
1. The probability that training sample $i$ is in the training set is $p_i = \frac{w_i}{\sum w_i}$ where the summation is over all members of the training set. Pick $n$ samples with replacement to form the training set.
2. Construct a regression machine $t$.
3. Make prediction $y_i^{(p)}(x_i)$ for $i=1,...,n$ with machine $t$. **Note: $y_i^{(p)}(x_i)$ is not the final prediction.**
4. Calculate a loss for $y_i^{(p)}(x_i)$ and $y_i$. The loss function may be of any functional form as long as $L \in$ [0,1]. If we let $$ D = sup|y_i^{(p)}(x_i)-y_i|  i = 1,...,n,$$ which means D is the largest error, then we have three candidate loss functions:
$$L_i=\frac{|y_i^{(p)}(x_i)-y_i|}{D} \textit{(linear)}$$
$$L_i=\frac{|y_i^{(p)}(x_i)-y_i|^2}{D^2} \textit{(square law)}$$
$$L_i=1-\exp\left[\frac{-|y_i^{(p)}(x_i)-y_i|}{D}\right] \textit{(exponential)}$$
5. Calculate aveage loss $\overline{L}=\sum_{i=1}^{N_1}L_ip_i$
6. Form $\beta=\frac{\overline{L}}{1-\overline{L}}$. $\beta$ is a measure of confidence in the predictor. Low $\beta$ means high confidence in the prediction.
7. Update the weights: $w_i \rightarrow w_i\beta^{[1-L_i]}$. The smaller the loss, the more weight is reduced , making the sample less likely to be picked in the next round.
8. For a particular input $x_i$, each of the $T$ machines makes a prediction $h_t, t=1,...,T$. $h_f$ is cumulative prediction using the $T$ predictors:
$$h_f = inf\bigg\{y \in Y: \sum_{t:h_t\leq y}log(\frac{1}{\beta_t})\geq\frac{1}{2}\sum_{t}log(\frac{1}{\beta_t})\bigg\}$$
This equation is essentially relabel $y_i$ such that $y_i^{(1)}<y_i^{(2)}<,...,y_i^{(T)}$. Then sum the $log(1/\beta_t)$ until we reach the smallest t that is equal or grater than $\frac{1}{2}\sum_{t}log(\frac{1}{\beta_t})$. This is the weighted median. If the $\beta_t$ were all equal, this would be the median.


In [322]:
from sklearn.ensemble import AdaBoostRegressor

In [322]:
abr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20), 
                        n_estimators=100,
                        loss='exponential')
abr.fit(X_train, np.ravel(y_train))
y_pred = abr.predict(X_test)
print(abr.score(X_test, y_test), np.sqrt(mean_squared_error(y_test, y_pred)))

In [322]:
param_grid = {'n_estimators': [100, 250, 500]}
abr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=10))
gs_cv = GridSearchCV(abr, param_grid, cv = 5)
gs_cv.fit(X_train, np.ravel(y_train))
print(gs_cv.best_params_, gs_cv.best_score_)

### Gradient Boosting  $_{[2]}$

#### AdaBoost vs Gradient Boosting
As oppose to AdaBoost which takes $\textit{the weighted median}$ of predictions from ensembled regressions, Gradient Boosting Model (GBM) is an $\textit{additive model}$ which adds up predicted values from ensembled regressions. 

#### General Form
The general form of gredient tree-boosting algorithm for regression could be expressed as
$$f(x) = \sum_{m=1}^{M}\beta_mb(x;\gamma_m),$$
where $\beta_m, m = 1,2,...,M$ are the expansion coefficients, and $b(x;\gamma)\in\mathbb{R}$ are usually simple functions of the multivariate argument $x$, characterized by a set of parameters $\gamma$. Specific algorithms are objtained by inserting different loss criteria $L(y, f(x))$

#### Gradient Tree Boosting Algorithm
1. Initialize $f_0(x) = \operatorname*{argmin}_\gamma\sum_{i=1}^{N} L(y_i,\gamma)$.
2. For $m = 1$ to $M$:
    
    (a) For $i = 1,2,...,N$ compute
    $$r_{im} = -\bigg[\frac{\partial L(y_i, f(x_i))}{\partial f(x_i)}\bigg]_{f=f_{m-1}}.$$
    
    $r$ is referred as generalized or $\textit{pseudo}$ residuals. If we set the loss function as $\frac{1}{2}[y_i - f(x_i)]^2$, the gradient will be $y_i - f(x_i)$. This step is also where the name of the model is from.
    
    (b) Fit a regression tree to target $r_{im}$ giving terminal regions $R_{jm}, j = 1,2,...,J_m$. 
    
    Here we fit $r$, not $y$. $J$ is the number of leafs. Each iteration $m$ might have different number of leaves $J_m$.
    
    (c) For $j=1,2,...,J_m$ compute
    $$\gamma_{jm} = \operatorname*{argmin}_\gamma\sum_{x_i\in R_{jm}}L(y_i, f_{m-1}X_i + \gamma).$$
    
    (d) Update $f_m(x) = f_{m-1}(x) + \sum_{j=1}^{J_m}\gamma_{jm}I(x\in R{jm})$.

3. Output $\hat{f}(x) = f_M(x)$.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
## Time it
gbr = GradientBoostingRegressor(learning_rate = 0.01,
                                n_estimators = 1000,
                                max_depth = 10)
gbr.fit(X_train, np.ravel(y_train))
y_pred = gbr.predict(X_test)
print(gbr.score(X_test, y_test), np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
param_grid = {'n_estimators': [100, 150],
              'learning_rate': np.linspace(0.1, 0.2, 10)}
gbr = GradientBoostingRegressor(max_depth=8, max_features='sqrt')
gs_cv = GridSearchCV(estimator=gbr, param_grid=param_grid, cv=5)
gs_cv.fit(X_train, np.ravel(y_train))
print(gs_cv.best_params_, gs_cv.best_score_)

### XGBoost $_{[3]}$
XGBoost is a scalable machine learning system for tree boosting. The systerm runs more than ten times faster than existing popular solutions on a single machine and scales to billions of examples in distributed or memory-limited setting. The phenomenal achievement is due to several innovative techniques, including:
- A highly scalable end-to-end tree boosting system.
- A theoretically justified weighted quantile sketch for efficient proposal calculation.
- A novel sparsity-aware algorithm for parallel tree learning.
- An effective cache-aware block structure for out-of-core tree learning.

Besides these major contributions, an additional improvements is the proposal of a regularized learning objective to prevent over fitting. This approach improves generalization from training data and increases prediction performance for regression, ranking, and classification problems.

#### Algorithm
**Note: The process below is simplified for a brief discussion. For more details please refer to the document [3] listed in references.**

The system implements gradient boosting, which performs additive optimization in functional space, and incorporates a regularized model to prevent over fitting.

**Gradient Boosting:**

$$\hat{y}_i = \phi(x_i) = \sum_{k=1}^{K}f_k(x_i), f_k\in\textit{F},$$

where $\textit{F}={f(x)=w_{q(x)}}(q:\mathbb{R}^m \rightarrow T, w \in \mathbb{R}^T)$ is the space of regression trees (also known as CART). Here $q$ represents the structure of each tree the maps an example to the corresponding leaf index. $T$ is the number of leaves in the tree. Each $f_k$ corresponds to an independent tree structure $q$ and leaf weight $w$. We use $w_i$ to represent score on $i$-th leaf.

**XGBoost then minimize the following loss function with $\textit{regularized}$ objective:** 

$$\textit{L}(\phi) = \sum_{i}l(\hat{y}_i, y_i)+\sum_{k}\Omega(f_k),$$
$$where \ \ \Omega(f) = \gamma T + \frac{1}{2}\lambda\|w\|^2$$

In addictive manner, let $\hat{y}_i^t$ be the prediction of the $i$-th instance at the $t$-th iteration, we will need to add $f_t$ to minimize:

$$\textit{L}^{(t)} = \sum_{i=1}^{n}l(y_i, \hat{y}_i^{t-1}+f_t(x_i))+\sum_{k}\Omega(f_k),$$

Apply second-order approximation to quickly optimize the objective and then remove the constant.
$$ $$
$$ $$

Define $I_j = {i|q(x_i)=j}$ as the instance set of leaf $j$ and expend $\omega$.
$$ $$

For a fixed structure $q(x)$, we can compute the optimal weight $w_j^*$ of leaf $j$ by
$$ $$

and calculate the corresponding optimal value by
$$ $$

Assume that $I_L$ and $I_R$ are the instance sets of left and right nodes after the split. Letting $I=I_L \union I_R$, then the loss reduction after the split is given by
$$ $$


## Performance on Kaggle

In [ ]:
X_test_kaggle, y_test_kaggle = house_price_data_cleaning(df_test, drop_cols)
lr_kaggle = LinearRegression()
lr_kaggle.fit(X, y)
y_pred_kaggle = lr_kaggle.predict(X_test_kaggle)
y_pred_kaggle

## References
1. H. Drucker, “Improving Regressors using Boosting Techniques”, 1997.
2. T. Hastie, R. Tibshirani and J. Friedman, Elements of Statistical Learning Ed. 2, Springer, 2009.
3. Tianqi Chen and Carlos Guestrin, "XGBoost: A Scalable Tree Boosting System", 2016.